In [1]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
import gradio as gr
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
# 📁 STEP 2: Mount Google Drive (if dataset is there)
from google.colab import drive
drive.mount('/content/drive')


IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/Brain Tumor Segmentation/Training'

CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']
data = []
labels = []

# ✅ Load and resize images
for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading {category}...")
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(category)
        except Exception as e:
            pass

# Preprocessing
data = np.array(data) / 255.0
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# Function to make predictions
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    label = lb.inverse_transform(prediction)
    return label[0]

# Create Gradio interface
interface = gr.Interface(fn=predict_image,
                         inputs=gr.Image(type='pil'),
                         outputs=gr.Label(num_top_classes=4),
                         title="Brain Tumor Classifier",
                         description="Upload an image to classify a brain tumor as glioma, meningioma, notumor, or pituitary.")
interface.launch()


Mounted at /content/drive
Loading glioma...
Loading meningioma...
Loading notumor...
Loading pituitary...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 204s 700ms/step - accuracy: 0.6229 - loss: 0.9129 - val_accuracy: 0.2616 - val_loss: 6.7857
Epoch 2/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 204s 713ms/step - accuracy: 0.7532 - loss: 0.6491 - val_accuracy: 0.4891 - val_loss: 1.6020
Epoch 3/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 204s 712ms/step - accuracy: 0.7796 - loss: 0.5607 - val_accuracy: 0.6570 - val_loss: 1.4711
Epoch 4/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 256s 690ms/step - accuracy: 0.8334 - loss: 0.4458 - val_accuracy: 0.4882 - val_loss: 1.3799
Epoch 5/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 208s 712ms/step - accuracy: 0.8455 - loss: 0.4360 - val_accuracy: 0.8163 - val_loss: 0.5905
Epoch 6/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 201s 704ms/step - accuracy: 0.8668 - loss: 0.3697 - val_accuracy: 0.5993 - val_loss: 1.6429
Epoch 7/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 196s 682ms/step - accuracy: 0.8587 - loss: 0.3741 - val_accuracy: 0.8539 - val_loss: 0.3677
Epoch 8/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 195s 682ms/step - accuracy: 0.8762 -